In [100]:
import pandas as pd
from rdkit import Chem
import requests
#from helpers.integration_helpers import canoicalize_smiles

## Carroll

In [113]:
carroll = pd.read_csv('../data/Carroll11.csv', encoding = "ISO-8859-1")

In [114]:
print(list(carroll))
#carroll = data[['Compound', 'TFP(K)Ref.']]
#carroll['source'] = 'carroll11'
#carroll['pure substance'] = 1
carroll.head()

['smiles', 'compound', 'flashpoint', 'source', 'pure substance']


,smiles,compound,flashpoint,source,pure substance
0,CCCCC,pentane,233.0,carroll11,1
1,CCCCCC,hexane,247.0,carroll11,1
2,CCCCCCC,heptane,269.0,carroll11,1
3,CCCCCCCC,octane,286.0,carroll11,1
4,CCCCCCCCC,nonane,304.0,carroll11,1


## Saldana

In [115]:
fuels = pd.read_csv('../data/fuels.csv', encoding = "ISO-8859-1")
fuels.head()

,smiles,compound,flashpoint,source,pure substance
0,OC/C=C/CO,"(2E)-but-2-ene-1,4-diol",401.0,Saldana11,1
1,OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,"(2R,3R,4R,5S)-hexane-1,2,3,4,5,6-hexol",422.0,Saldana11,1
2,OCC[C@H](O)CO,"(2S)-butane-1,2,4-triol",385.0,Saldana11,1
3,OC/C=C\CO,"(2Z)-but-2-ene-1,4-diol",401.0,Saldana11,1
4,OCCCCCCCC/C=C\C/C=C\C/C=C\CC,"(9Z,12Z,15Z)-octadeca-9,12,15-trien-1-ol",NaN,Saldana11,1


## Pan12

In [116]:
pan12 = pd.read_csv('../data/pan12.csv', encoding = "ISO-8859-1")
pan12.head()

,smiles,compound,flashpoint,source,pure substance
0,Cl[SiH](Cl)Cl,Trichlorosilane,245.0,pan12,1
1,Cl[SiH2]Cl,Dichlorosilane,236.0,pan12,1
2,C[Si](Cl)(Cl)Cl,Methyltrichlorosilane,276.0,pan12,1
3,C[SiH](Cl)Cl,Methyldichlorosilane,241.0,pan12,1
4,ClC(Cl)[Si](Cl)(Cl)C,Dichloromethylmethyldichlorosilane,301.0,pan12,1


## chen14

In [117]:
chen14 = pd.read_csv('../data/chen14.csv', encoding = "ISO-8859-1")

In [118]:
chen14.head()

,smiles,compound,flashpoint,source,pure substance
0,Cl[SiH](Cl)Cl,Trichlorosilane,245.15,chen14,1
1,C[Si](Cl)(Cl)Cl,Methyltrichlorosilane,276.15,chen14,1
2,C(=C)[Si](Cl)(Cl)Cl,Vinyltrichlorosilane,294.15,chen14,1
3,ClC(Cl)[Si](Cl)(Cl)C,Dichloromethylmethyldichlorosilane,301.15,chen14,1
4,C(C)[Si](Cl)(Cl)Cl,Ethyltrichlorosilane,263.15,chen14,1


In [ ]:
# canonicalize carroll smiles
for i in range(0, len(carroll)):
    carroll.iloc[i]['smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(carroll.iloc[i]['smiles']))


In [ ]:
#canonicalize smiles with rdkit
result['canonical smiles'] = None
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.iloc[idx]['smiles']))

In [ ]:
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.loc[idx]['smiles']))

## Add Gelest to dataset

In [119]:
silanes = pd.read_csv('../data/silanes_all.csv')
metal_organic = pd.read_csv('../data/metal-organics_all.csv')
tin = pd.read_csv('../data/tin_all.csv')
germanium  = pd.read_csv('../data/germanium_all.csv')
print(list(silanes))
print(list(metal_organic))
print(list(tin))
print(list(germanium))

['compound', 'smiles', 'flashpoint', 'pure substance', 'source']
['compound', 'smiles', 'flashpoint', 'pure substance', 'source']
['compound', 'smiles', 'flashpoint', 'pure substance', 'source']
['compound', 'smiles', 'flashpoint', 'pure substance', 'source']


In [125]:
#result = result[['compound', 'smiles', 'flashpoint', 'pure substance', 'source']]
silanes_pure = silanes[silanes['pure substance'] == 1]
metal_organic_pure = metal_organic[metal_organic['pure substance'] == 1]
tin_pure = tin[tin['pure substance'] == 1]
germanium_pure = germanium[germanium['pure substance'] == 1]

## pubchem integration

In [147]:
pubchem = pd.read_csv('../data/pubchem_data.csv')
pubchem = pubchem[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
germanium_pure = germanium_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
tin_pure = tin_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
metal_organic_pure = metal_organic_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
silanes_pure = silanes_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
hazardous_mat = hazardous_mat[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]


In [148]:
print(list(pubchem))
print(list(carroll))
print(list(chen14))
print(list(fuels))
print(list(germanium_pure))
print(list(metal_organic_pure))
print(list(pan12))
print(list(silanes_pure))
print(list(tin_pure))
print(list(hazardous_mat))

['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']


In [151]:
def canoicalize_smiles(data):
    for idx, row in data.iterrows():
        m = Chem.MolFromSmiles(data.iloc[idx]['smiles'])
        if m != None:
            data.iloc[idx]['smiles'] = Chem.MolToSmiles(m)
        else:
            data.iloc[idx]['smiles'] = None
    return data

In [149]:
frames = [pubchem, carroll, chen14, fuels, germanium_pure, metal_organic_pure, pan12, silanes_pure, tin_pure, hazardous_mat]
result = pd.concat(frames)

In [152]:
canonical_smiles_data = canoicalize_smiles(result)

C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [160]:
print(canonical_smiles_data.shape)
print(canonical_smiles_data.dropna().shape)
canonical_smiles_data.dropna(inplace=True)

(4810, 5)
(4282, 5)


In [161]:
canonical_smiles_data.to_csv('../data/integrated_dataset.csv', index=False)

In [59]:
def remove_duplicates(data):
    duplicates = data[data.duplicated(subset='smiles')]
    result = data.drop_duplicates(subset='smiles', keep=False)#[~duplicates]
    #for each unique smiles that has duplicates
    for smiles in data[data.duplicated(subset='smiles')]['smiles'].unique():
        dup_rows = data.loc[data['smiles'] == smiles]
        if dup_rows['flashpoint'].unique().shape[0] == 1:
            # remove all but one
            result = result.append(dup_rows.iloc[0], sort=False)
        else:
            if dup_rows['flashpoint'].std() < 5:
                # add 1 back
                result = result.append(dup_rows.iloc[0], sort=False)
    return result  
            

In [ ]:
pure_compounds = data.loc[data['pure substance'] == 1]
pure_compounds.shape

In [ ]:
data[data.duplicated(subset='smiles')].shape

In [ ]:
data = data.loc[data['pure substance'] == 1]
print(data.shape)
no_dups = remove_duplicates(data)
print(no_dups.shape)

In [ ]:
no_dups[no_dups.duplicated(subset='smiles')]
#no_dups.loc[no_dups['smiles'] == 'CC1=CCCC(=C)C2CC(C2CC1)(C)C']
no_dups.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
result = data.drop_duplicates(subset='smiles', keep=False)
result.shape

In [ ]:
data[data.duplicated(subset='smiles')]['smiles'].unique()

### Hazardous Materials

In [135]:
data = pd.read_csv('../data/hazardous_materials.csv')
haz = data[['Common name', 'Data type', 'Measurement name', 'Measurement value', 'Measurement units']]

In [136]:
flashpoints = haz.loc[haz['Measurement name'] == 'Flash point']

In [137]:
print(list(flashpoints))

['Common name', 'Data type', 'Measurement name', 'Measurement value', 'Measurement units']


In [138]:
flashpoints['smiles'] = flashpoints['Common name'].apply(get_smiles)

C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [139]:
def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

def get_smiles(compound):
    smilesUrl = "https://opsin.ch.cam.ac.uk/opsin/"+compound+".smi"
    requestSmiles = requests.get(smilesUrl)
    if requestSmiles.status_code == 400 or requestSmiles.status_code == 404:
        return None
    else:
        return Chem.MolToSmiles(Chem.MolFromSmiles(requestSmiles.text))  
    
def celsiusToKelvin(temp):
    return temp + 273.15

def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

In [140]:
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('<')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('>')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('~')]
pattern = '\\w+-\\w'
flashpoints['Measurement value'] = flashpoints['Measurement value'].str.replace(pattern, 'none', regex=True)
#flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('none')]
flashpoints['Measurement value'] = pd.to_numeric(flashpoints['Measurement value'], errors='coerce')

flashpoints.head()

,Common name,Data type,Measurement name,Measurement value,Measurement units,smiles
2,Acetal,Experimental,Flash point,-21.0,$^{\circ}$C,None
11,Acetaldehyde,Experimental,Flash point,-38.0,$^{\circ}$C,CC=O
18,Acetanilide,Experimental,Flash point,174.0,$^{\circ}$C,CC(=O)Nc1ccccc1
26,Acetic acid,Experimental,Flash point,43.0,$^{\circ}$C,CC(=O)O
35,Acetic anhydride,Experimental,Flash point,54.0,$^{\circ}$C,CC(=O)OC(C)=O


In [141]:
flashpoints['flashpoint'] = flashpoints['Measurement value'].apply(celsiusToKelvin)
flashpoints.drop(columns=['Measurement name','Data type', 'Measurement units', 'Measurement value'], inplace=True)
#flashpoints.rename(columns={'Common name' : 'compound', 'Measurement value' : 'flashpoint'}, inplace=True)

In [142]:
flashpoints.dropna(inplace=True)
flashpoints.rename(columns={'Common name' : 'compound'}, inplace=True)
#flashpoints.drop(df.columns[i], axis=1)
flashpoints['pure substance'] = 1
flashpoints['source'] = 'hazardous materials'
hazardous_mat = flashpoints[['compound', 'flashpoint', 'pure substance', 'smiles', 'source']]
hazardous_mat.shape

(368, 5)

In [143]:
hazardous_mat.head()

,compound,flashpoint,pure substance,smiles,source
11,Acetaldehyde,235.15,1,CC=O,hazardous materials
18,Acetanilide,447.15,1,CC(=O)Nc1ccccc1,hazardous materials
26,Acetic acid,316.15,1,CC(=O)O,hazardous materials
35,Acetic anhydride,327.15,1,CC(=O)OC(C)=O,hazardous materials
44,Acetone,255.15,1,CC(C)=O,hazardous materials


In [61]:
#data = pd.read_csv('../data/integrated_dataset.csv')
#frames = [data, hazardous_mat]
#result = pd.concat(frames)
no_dups = remove_duplicates(result)
print(result.shape)
print(no_dups.shape)
#no_dups.to_csv('../data/integrated_dataset.csv', index=False)

(3684, 5)
(3471, 5)
